In [1]:
import torch
from model import *
from model2 import Encoder_decoder
import model
from utils import Combined_Loss, Combined_Loss2
from data_loader import VAEDataset, VAEDataset1
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchsummary import summary
import time

In [2]:
HALF = False
PIN_MEM = True
BATCH_SIZE_TRAIN = 2
BATCH_SIZE_TEST = BATCH_SIZE_TRAIN
NUM_WORKERS = 0
LOAD = True
DEPTH = 5
train_save = open("log/train_loss.txt", "a")
test_save = open("log/test_loss.txt", "a")

model = Encoder_decoder(DEPTH, 1, 3).cuda()
if LOAD:
    model.load_state_dict(torch.load("w2_edof.pth"))
    t = model.eval()
else:
    t = model.train()
#rgb_p = 'archive/FLIR_ADAS_1_3/train/RGB/'
#ir__p = 'archive/FLIR_ADAS_1_3/train/thermal_8_bit/'

rgb_p = 'archive/dataset/files/img_raw/rgb/'
ir__p = 'archive/dataset/files/img_raw/nir/'

train_ds = VAEDataset1(rgb_p, ir__p, half=HALF)
train_loader = DataLoader(
        dataset=train_ds,
        batch_size=BATCH_SIZE_TRAIN,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEM,
        shuffle=True
    )

#rgb_vp = 'archive/FLIR_ADAS_1_3/val/RGB/'
#ir__vp = 'archive/FLIR_ADAS_1_3/val/thermal_8_bit/'
rgb_vp = 'archive/dataset/files/img_raw/rgb/'
ir__vp = 'archive/dataset/files/img_raw/nir/'

val_ds = VAEDataset1(rgb_vp, ir__vp, half=HALF)
val_loader = DataLoader(
        dataset=val_ds,
        batch_size=BATCH_SIZE_TEST,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEM,
        shuffle=True
    )


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss2 = Combined_Loss2()
#summary(model, [(1,256,256), (3,256,256)])
def pyramid(ir, rgb):
    ir, rgb = [ir.cuda()], [rgb.cuda()]
    for i in range(1, DEPTH):
        ir.append(nn.functional.interpolate(ir[-1], scale_factor=0.5).cuda())
        rgb.append(nn.functional.interpolate(rgb[-1], scale_factor=0.5).cuda())
    return ir, rgb

In [3]:
for epc in range(54,200):
    epoch_loss = []
    r = tqdm(enumerate(train_loader), leave=False, desc="Epoch: {}".format(epc), total=len(train_loader))
    for idx, (ir, rgb) in r:
        ir, rgb = pyramid(ir, rgb)
        If = model(ir, rgb)
        lf = loss2.compute(ir[0], rgb[0], If)
        epoch_loss.append(lf.item())
        del rgb, ir, If
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        lf.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=1)
        optimizer.step()
        r.set_postfix(loss=lf.item())
    #save epoch loss list
    train_save.write("epoch [{}] = ".format(epc) + str(epoch_loss) + '\n')
    #save checkpoint
    torch.save(model.state_dict(), "w2_edof.pth")
    
    #test loss run on val
    with torch.no_grad():
        e = model.eval()
        ir, rgb = next(iter(val_loader))
        ir, rgb = pyramid(ir, rgb)
        If = model(ir, rgb)
        lf = loss2.compute(ir[0], rgb[0], If)
        print("epoch [{}] = ".format(epc) + str(float(lf)))
        test_save.write("epoch [{}] = ".format(epc) + str(float(lf)) + '\n')
        
        del rgb, ir, If
        torch.cuda.empty_cache()

epoch [54] = 0.05231828987598419


epoch [55] = 0.05423462763428688


epoch [56] = 0.07036499679088593


epoch [57] = 0.04989739507436752


epoch [58] = 0.04707597941160202


...

epoch [160] = 0.053597502410411835


epoch [161] = 0.06437430530786514


epoch [162] = 0.04051556810736656


epoch [163] = 0.05401865392923355


KeyboardInterrupt: 